# Module 3. Personalize 캠페인 생성 하기

이 노트북은 Module2에서 생성한 솔류션을 바탕으로 아래와 같은 작업을 합니다.
* 캠페인 생성
* 캠페인을 통해 특정 유저에 대한 추천 영화 리스트 얻기


## 라이브러리 임포트

파이썬에는 광범위한 라이브러리 모음이 포함되어 있으며, 본 핸즈온을 위해서 핵심 데이터 과학 도구인 boto3 (AWS SDK) 및 Pandas/Numpy와 같은 라이브러리를 가져와야 합니다.

In [1]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time

다음으로 여러분의 환경이 Amazon Personalize와 성공적으로 통신할 수 있는지 확인해야 합니다.

In [2]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

아래 코드 셀은 이전 notebook에서 저장했던 공유 변수들을 불러옵니다.

In [23]:
%store -r


생성할 오브젝트의 끝에 임의의 숫자를 부여하기 위해 suffix 정의

In [4]:
suffix = str(np.random.uniform())[4:9]

### 캠페인 생성 및 대기

작동하는 솔루션 버전을 보유하고 있으므로, 이제 애플리케이션과 함께 사용할 캠페인을 작성해야 합니다. 캠페인은 단순히 모델의 호스팅된 사본입니다. 물론 인프라가 프로비저닝되기까지의 시간이 소요됩니다.

#### 캠페인 생성

In [5]:
create_campaign_response = personalize.create_campaign(
    name = "DEMO-hrnn-campaign" + suffix,
    solutionVersionArn = hrnn_solution_version_arn,
    minProvisionedTPS = 1
)

hrnn_campaign_arn = create_campaign_response['campaignArn']
print(json.dumps(create_campaign_response, indent=2))

{
  "campaignArn": "arn:aws:personalize:ap-northeast-2:870180618679:campaign/DEMO-hrnn-campaign75412",
  "ResponseMetadata": {
    "RequestId": "69bb0a37-3601-4a24-99a5-674883fdee01",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Sun, 07 Jun 2020 06:51:19 GMT",
      "x-amzn-requestid": "69bb0a37-3601-4a24-99a5-674883fdee01",
      "content-length": "98",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [7]:
create_campaign_response = personalize.create_campaign(
    name = "DEMO-hrnn-coldstart-campaign-" + suffix,
    solutionVersionArn = hrnn_coldstart_solution_version_arn,
    minProvisionedTPS = 1
)

hrnn_coldstart_campaign_arn = create_campaign_response['campaignArn']
print(json.dumps(create_campaign_response, indent=2))

{
  "campaignArn": "arn:aws:personalize:ap-northeast-2:870180618679:campaign/DEMO-hrnn-coldstart-campaign-75412",
  "ResponseMetadata": {
    "RequestId": "1faf97fd-d0ce-442c-a513-09b0f39420f6",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Sun, 07 Jun 2020 06:51:31 GMT",
      "x-amzn-requestid": "1faf97fd-d0ce-442c-a513-09b0f39420f6",
      "content-length": "109",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [8]:
create_campaign_response = personalize.create_campaign(
    name = "DEMO-sims-campaign-" + suffix,
    solutionVersionArn = sims_solution_version_arn,
    minProvisionedTPS = 1
)

sims_campaign_arn = create_campaign_response['campaignArn']
print(json.dumps(create_campaign_response, indent=2))

{
  "campaignArn": "arn:aws:personalize:ap-northeast-2:870180618679:campaign/DEMO-sims-campaign-75412",
  "ResponseMetadata": {
    "RequestId": "68f3e219-697e-4553-98f9-7d876f6c4be7",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Sun, 07 Jun 2020 06:51:35 GMT",
      "x-amzn-requestid": "68f3e219-697e-4553-98f9-7d876f6c4be7",
      "content-length": "99",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [9]:
create_campaign_response = personalize.create_campaign(
    name = "DEMO-ranking-campaign-" + suffix,
    solutionVersionArn = ranking_solution_version_arn,
    minProvisionedTPS = 1
)

ranking_campaign_arn = create_campaign_response['campaignArn']
print(json.dumps(create_campaign_response, indent=2))

{
  "campaignArn": "arn:aws:personalize:ap-northeast-2:870180618679:campaign/DEMO-ranking-campaign-75412",
  "ResponseMetadata": {
    "RequestId": "5c477024-d909-4141-a71e-bc9da337dca1",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Sun, 07 Jun 2020 06:51:36 GMT",
      "x-amzn-requestid": "5c477024-d909-4141-a71e-bc9da337dca1",
      "content-length": "102",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### 캠페인이 활성화 상태가 될 때까지 대기
소요시간은 약 20분 걸립니다.

In [40]:
%%time

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = hrnn_campaign_arn
    )
    status_hrnn = describe_campaign_response["campaign"]["status"]
    print("HRNN_Campaign: {}".format(status_hrnn))
    
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = hrnn_coldstart_campaign_arn
    )
    status_hrnn_cs = describe_campaign_response["campaign"]["status"]
    print("HRNN_Coldstart_Campaign: {}".format(status_hrnn_cs))
    
    describe_campaign_response = personalize.describe_campaign(
    campaignArn = sims_campaign_arn
    )
    status_sims = describe_campaign_response["campaign"]["status"]
    print("Sims_Campaign: {}".format(status_sims))
 
    describe_campaign_response = personalize.describe_campaign(
    campaignArn = ranking_campaign_arn
    )
    status_ranking = describe_campaign_response["campaign"]["status"]
    print("Ranking_Campaign: {}".format(status_ranking))
    
        
    
    if (status_hrnn == "ACTIVE" or status_hrnn == "CREATE FAILED")&\
       (status_hrnn_cs == "ACTIVE" or status_hrnn_cs == "CREATE FAILED")&\
       (status_sims == "ACTIVE" or status_sims == "CREATE FAILED")&\
       (status_ranking == "ACTIVE" or status_ranking == "CREATE FAILED"):
        break
    print("-------------------------------------->")
    time.sleep(60)

print("All Campaign creation completed")   

HRNN_Campaign: ACTIVE
HRNN_Coldstart_Campaign: ACTIVE
Sims_Campaign: ACTIVE
Ranking_Campaign: ACTIVE
All Campaign creation completed
CPU times: user 19.4 ms, sys: 0 ns, total: 19.4 ms
Wall time: 185 ms


## 샘플 추천 결과 얻기

캠페인이 활성화되면 추천 결과를 받을 수 있습니다. 먼저 컬렉션에서 임의의 사용자를 선택해야 합니다. 그런 다음, ID 대신 추천을 위해 영화 정보를 표시하는 몇 가지 헬퍼 함수를 만듭니다.

In [92]:
df=pd.read_csv(interaction_filename)

# Getting a random user:
user_id, item_id, _,_,_ = df.sample().values[0]
print("USER: {}".format(user_id))

USER: 4205


In [93]:
items_all = pd.read_csv('./ml-1m/movies.dat',sep='::', encoding='latin1',names=['ITEM_ID', 'TITLE', 'GENRE'],)
items=items_all.copy()
items['to_keep'] = items['ITEM_ID'].apply(lambda x:x in unique_items)
items=items[items['to_keep']]
#items=items.set_index('ITEM_ID')
del items['to_keep']
items.tail()

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':


ITEM_ID                       TITLE           GENRE
3878     3948     Meet the Parents (2000)          Comedy
3879     3949  Requiem for a Dream (2000)           Drama
3880     3950            Tigerland (2000)           Drama
3881     3951     Two Family House (2000)           Drama
3882     3952       Contender, The (2000)  Drama|Thriller

In [94]:
def get_movie_title(movie_id):
    """
    Takes in an ID, returns a title
    """
    movie_id = int(movie_id)
    movie_title=items[items['ITEM_ID']==movie_id]['TITLE']
    return (movie_title.tolist())


#### GetRecommendations 호출

아래 코드 셀을 실행하면 특정 사용자에 대한 추천 사항이 표시되고 추천 영화 목록이 반환됩니다.

In [95]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = hrnn_campaign_arn,
    userId = str(user_id),
)
# Update DF rendering
pd.set_option('display.max_rows', 30)

print("Recommendations for user: ", user_id)

item_list = get_recommendations_response['itemList']

recommendation_list = []

for item in item_list:
    title = get_movie_title(item['itemId'])
    recommendation_list.append(title)
    
recommendations_df = pd.DataFrame(recommendation_list, columns = ['OriginalRecs'])
recommendations_df

Recommendations for user:  4205


OriginalRecs
0                  NeverEnding Story, The (1984)
1                            Time Bandits (1981)
2                                    Tron (1982)
3                                  Willow (1988)
4                            Goonies, The (1985)
5                       Dark Crystal, The (1982)
6                                    Dune (1984)
7               7th Voyage of Sinbad, The (1958)
8   Willy Wonka and the Chocolate Factory (1971)
9     Navigator: A Mediaeval Odyssey, The (1988)
10               Who Framed Roger Rabbit? (1988)
11           20,000 Leagues Under the Sea (1954)
12                            Beetlejuice (1988)
13                                 Splash (1984)
14                                    Big (1988)
15                               Superman (1978)
16                                Starman (1984)
17                            Heavy Metal (1981)
18                 Boy Who Could Fly, The (1986)
19            Evil Dead II (Dead By Dawn) (1987)
20                           Return to Oz (1985)
21                             Highlander (1986)
22                             Abyss, The (1989)
23                         Watership Down (1978)
24                                 Legend (1985)

## 리뷰

캠페인을 생성하고 실제적으로 특정 유저의 추천 영화 목록도 얻었습니다.
이제 다음 노트북으로 넘어갈 준비가 되었습니다. (`4.View_Campaign_And_Interactions.ipynb`)


## 다음 노트북에 대한 참고 사항

다음 실습에 필요한 몇 가지 값들이 있습니다. 아래 셀을 실행하여 저장한 후, 다음 주피터 노트북에서 그대로 사용할 수 있습니다.

In [99]:
%store hrnn_campaign_arn
%store hrnn_coldstart_campaign_arn
%store sims_campaign_arn
%store recommendations_df
%store user_id

Stored 'hrnn_campaign_arn' (str)
Stored 'hrnn_coldstart_campaign_arn' (str)
Stored 'sims_campaign_arn' (str)
Stored 'recommendations_df' (DataFrame)
Stored 'user_id' (int)
